In [14]:
import tensorflow_datasets as tfds
import numpy as np


import pandas as pd

In [2]:
ds, info = tfds.load("lfw", with_info=True, download=True, as_supervised=False)

2024-01-23 16:29:39.369837: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-23 16:29:39.408207: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-23 16:29:39.408231: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-23 16:29:39.409093: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-23 16:29:39.414773: I tensorflow/core/platform/cpu_feature_guar

In [3]:
def read_pairs(pairs_filename):
    pairs = ([], [])
    with open(pairs_filename, "r") as f:
        nPairs = int(f.readline())
        for i in range(2):
            for _ in range(nPairs):
                pair = tuple(next(f).strip().split())
                pairs[i].append(pair)
    # print(len(pairs[0]), len(pairs[1]))
    return np.array(pairs[0]), np.array(pairs[1])

In [4]:
def expand_filenames(pos_set: np.array, neg_set: np.array) -> np.array:
    expand = []
    for pair in pos_set:
        expand.append(
            (
                pair[0] + f"_{int(pair[1]):04d}.jpg",
                pair[0] + f"_{int(pair[2]):04d}.jpg",
                1,
            )
        )
    for pair in neg_set:
        expand.append(
            (
                pair[0] + f"_{int(pair[1]):04d}.jpg",
                pair[2] + f"_{int(pair[3]):04d}.jpg",
                0,
            )
        )

    return np.array(expand)

In [5]:
def img_pair_df(pairs_filename):
    pairs = read_pairs(pairs_filename)
    print(
        f"Positive Pairs:{pairs[0].shape[0]}", f"\nNegative Pairs:{pairs[1].shape[0]}"
    )
    pairs = expand_filenames(pairs[0], pairs[1])
    return pd.DataFrame(pairs, columns=["img_0", "img_1", "pair"])

In [6]:
pos, neg = read_pairs("pairsDevTest.txt")

In [7]:
pd.DataFrame(expand_filenames(pos, neg), columns=["img_0", "img_1", "pair"])

,img_0,img_1,pair
0,Abdullah_Gul_0013.jpg,Abdullah_Gul_0014.jpg,1
1,Abdullah_Gul_0013.jpg,Abdullah_Gul_0016.jpg,1
2,Abdullatif_Sener_0001.jpg,Abdullatif_Sener_0002.jpg,1
3,Adel_Al-Jubeir_0001.jpg,Adel_Al-Jubeir_0003.jpg,1
4,Al_Pacino_0001.jpg,Al_Pacino_0002.jpg,1
...,...,...,...
995,Susan_Whelan_0001.jpg,Wolfgang_Schneiderhan_0001.jpg,0
996,Takeo_Fukui_0001.jpg,Will_Ofenheusle_0001.jpg,0
997,Tamara_Mowry_0001.jpg,Zach_Parise_0001.jpg,0
998,Tatiana_Kennedy_Schlossberg_0001.jpg,Thomas_Watjen_0001.jpg,0


In [8]:
img_pair_df("pairsDevTest.txt")

Positive Pairs:500 
Negative Pairs:500


,img_0,img_1,pair
0,Abdullah_Gul_0013.jpg,Abdullah_Gul_0014.jpg,1
1,Abdullah_Gul_0013.jpg,Abdullah_Gul_0016.jpg,1
2,Abdullatif_Sener_0001.jpg,Abdullatif_Sener_0002.jpg,1
3,Adel_Al-Jubeir_0001.jpg,Adel_Al-Jubeir_0003.jpg,1
4,Al_Pacino_0001.jpg,Al_Pacino_0002.jpg,1
...,...,...,...
995,Susan_Whelan_0001.jpg,Wolfgang_Schneiderhan_0001.jpg,0
996,Takeo_Fukui_0001.jpg,Will_Ofenheusle_0001.jpg,0
997,Tamara_Mowry_0001.jpg,Zach_Parise_0001.jpg,0
998,Tatiana_Kennedy_Schlossberg_0001.jpg,Thomas_Watjen_0001.jpg,0


In [9]:
df = img_pair_df("pairsDevTrain.txt")

Positive Pairs:1100 
Negative Pairs:1100


,img_0,img_1,pair
0,Aaron_Peirsol_0001.jpg,Aaron_Peirsol_0002.jpg,1
1,Aaron_Peirsol_0003.jpg,Aaron_Peirsol_0004.jpg,1
2,Aaron_Sorkin_0001.jpg,Aaron_Sorkin_0002.jpg,1
3,Abdel_Nasser_Assidi_0001.jpg,Abdel_Nasser_Assidi_0002.jpg,1
4,Abdullah_0001.jpg,Abdullah_0003.jpg,1
...,...,...,...
2195,Tom_Vilsack_0001.jpg,Wayne_Ferreira_0005.jpg,0
2196,Trisha_Meili_0001.jpg,Vladimiro_Montesinos_0003.jpg,0
2197,Ty_Votaw_0001.jpg,Wayne_Allard_0001.jpg,0
2198,Vytas_Danelius_0001.jpg,Zaini_Abdullah_0001.jpg,0


In [23]:
ds = tfds.load("lfw_pairs")

In [41]:
tfds.as_dataframe(ds["train"].take(5))

2024-01-23 17:59:47.217199: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


,img_0,img_1,pair
0,[[[ 88 90 103] [ 88 90 103] [ 88 90 103] ... [111 107 95] [119 115 104] [120 118 106]] [[ 89 91 104] [ 89 91 104] [ 88 90 103] ... [112 106 94] [116 112 101] [118 116 104]] [[ 90 92 105] [ 90 92 105] [ 89 91 104] ... [111 105 93] [114 110 99] [116 112 101]] ... [[ 46 44 45] [ 46 44 45] [ 46 44 45] ... [ 41 39 42] [ 42 40 43] [ 42 40 43]] [[ 46 44 45] [ 46 44 45] [ 46 44 45] ... [ 40 38 41] [ 42 40 43] [ 42 40 43]] [[ 46 44 45] [ 46 44 45] [ 46 44 45] ... [ 40 38 41] [ 42 40 43] [ 42 40 43]]],[[[44 39 19] [45 40 21] [43 34 19] ... [36 34 19] [36 34 21] [36 34 21]] [[41 36 16] [44 39 20] [46 37 22] ... [36 34 19] [35 33 20] [35 33 20]] [[39 34 14] [44 39 20] [48 39 24] ... [36 34 19] [35 33 20] [35 33 20]] ... [[29 28 24] [29 28 24] [28 29 24] ... [52 52 52] [52 52 52] [52 52 52]] [[28 27 23] [28 27 23] [27 28 23] ... [52 52 52] [52 52 52] [52 52 52]] [[28 27 23] [28 27 23] [27 28 23] ... [52 52 52] [52 52 52] [52 52 52]]],0
1,[[[ 0 0 0] [ 0 0 0] [ 0 0 0] ... [ 0 0 0] [ 0 0 0] [ 0 0 0]] [[ 0 0 0] [ 0 0 0] [ 0 0 0] ... [ 0 0 0] [ 0 0 0] [ 0 0 0]] [[ 0 0 0] [ 0 0 0] [ 0 0 0] ... [ 0 1 0] [ 0 1 0] [ 0 1 0]] ... [[ 0 0 0] [ 0 0 0] [ 0 0 0] ... [17 18 22] [20 21 25] [20 21 25]] [[ 0 0 0] [ 0 0 0] [ 0 0 0] ... [18 19 21] [18 19 23] [24 25 29]] [[ 0 0 0] [ 0 0 0] [ 0 0 0] ... [16 17 19] [18 19 23] [24 25 29]]],[[[ 0 0 0] [ 0 0 0] [ 0 0 0] ... [173 32 14] [155 36 14] [197 88 65]] [[ 0 0 0] [ 0 0 0] [ 0 0 0] ... [169 28 10] [166 45 24] [216 107 84]] [[ 0 0 0] [ 0 0 0] [ 0 0 0] ... [168 23 4] [181 60 39] [244 134 111]] ... [[ 0 0 0] [ 0 0 0] [ 0 0 0] ... [ 14 13 9] [ 21 17 14] [ 22 18 15]] [[ 0 0 0] [ 0 0 0] [ 0 0 0] ... [ 16 17 12] [ 20 19 15] [ 20 19 15]] [[ 0 0 0] [ 0 0 0] [ 0 0 0] ... [ 11 12 7] [ 20 19 15] [ 20 19 15]]],0
2,[[[61 26 4] [69 35 10] [77 40 14] ... [67 31 15] [67 32 13] [67 32 13]] [[64 29 7] [68 34 9] [75 38 12] ... [63 27 11] [65 30 11] [66 31 12]] [[67 33 8] [68 34 9] [73 36 10] ... [59 23 7] [63 28 9] [63 28 9]] ... [[63 31 10] [62 30 9] [61 29 8] ... [82 79 60] [82 79 62] [81 78 61]] [[61 28 9] [62 29 10] [59 28 8] ... [86 83 64] [86 83 66] [80 77 60]] [[61 28 9] [61 28 9] [58 27 9] ... [86 83 64] [87 84 67] [81 78 61]]],[[[ 0 0 0] [ 0 0 0] [ 0 0 0] ... [ 0 0 0] [ 0 0 0] [ 0 0 0]] [[ 0 0 0] [ 0 0 0] [ 0 0 0] ... [ 0 0 0] [ 0 0 0] [ 0 0 0]] [[ 0 0 0] [ 0 0 0] [ 0 0 0] ... [ 0 0 0] [ 0 0 0] [ 0 0 0]] ... [[10 0 0] [62 46 30] [64 41 27] ... [27 26 22] [26 25 23] [26 25 23]] [[11 0 0] [61 45 29] [62 38 26] ... [27 26 22] [20 19 17] [22 21 19]] [[11 0 0] [61 45 29] [62 38 26] ... [21 20 16] [21 20 18] [23 22 20]]],1
3,[[[ 0 0 0] [ 0 0 0] [ 0 0 0] ... [ 0 0 0] [ 0 0 0] [ 0 0 0]] [[ 0 0 0] [ 0 0 0] [ 0 0 0] ... [ 0 0 0] [ 0 0 0] [ 0 0 0]] [[ 0 1 0] [ 0 1 0] [ 0 1 0] ... [ 0 0 0] [ 0 0 0] [ 0 0 0]] ... [[152 159 178] [150 157 176] [146 153 172] ... [140 143 148] [117 120 125] [ 79 83 86]] [[136 143 161] [135 142 160] [135 142 160] ... [108 109 114] [ 67 68 72] [ 55 56 58]] [[136 143 161] [133 140 158] [131 138 156] ... [ 67 68 72] [ 49 50 54] [ 40 41 43]]],[[[165 151 140] [166 152 141] [167 153 142] ... [176 163 157] [175 162 156] [175 162 156]] [[165 149 136] [164 150 137] [167 151 138] ... [176 163 157] [175 162 156] [175 162 156]] [[165 147 133] [164 148 133] [166 148 134] ... [175 162 156] [176 163 157] [176 163 157]] ... [[174 201 244] [175 202 245] [175 202 245] ... [ 72 50 39] [ 80 60 49] [ 76 56 45]] [[174 201 244] [174 201 244] [173 200 245] ... [ 63 41 30] [ 69 49 38] [ 75 55 44]] [[171 198 241] [171 198 241] [172 199 244] ... [ 59 37 26] [ 65 45 34] [ 70 50 39]]],0
4,[[[ 0 0 0] [ 0 0 0] [ 0 0 0] ... [ 0 1 2] [ 0 0 2] [ 0 0 2]] [[ 0 0 0] [ 0 0 0] [ 0 0 0] ... [ 0 1 0] [ 0 0 2] [ 0 1 0]] [[ 0 0 0] [ 0 0 0] [ 0 0 0] ... [ 0 1 0] [ 0 1 0] [ 0 1 0]] ... [[ 0 0 0] [ 0 0 0] [ 0 0 0] ... [48 28 1] [59 39 12] [63 43 16]] [[ 0 0 0] [ 0 0 0] [ 0 0 0] ... [51 30 3] [63 42 15] [70 49 22]] [[ 0 0 0] [ 0 0 0] [ 0 0 0] ... [59 38 11] [67 46 19] [75 54 27]]],[[[ 0 2 0] [ 0 2 0] [ 0 1 0] ... [ 0 0 0] [ 0 0 0] [